In [312]:
from clarifai.rest import ClarifaiApp
app = ClarifaiApp(api_key='97246cf0da164fd9963b504cccaca5ba')

In [316]:
from clarifai.rest import Image as CImage
import json



# this function returns the ingredients queried on clarifai
def get_ingreds(img_path,url_flag=1):
    model = app.models.get('food-items-v1.0')
    
    image = None
    if(not url_flag):
        image = CImage(file_obj=open(img_path,'rb'))
    else:
        image = CImage(url=img_path)
        
    response = model.predict([image])
    parsed_response = json.loads(json.dumps(response))

    json_dict = parsed_response['outputs'][0]['data']['concepts']

    for i in json_dict:
        print(i)
        
    return json.dumps(json_dict)


# this function returns matching recipe names for the given response 

def get_matching_recipes(json_resp, recipes, url_flag = 1):
    
    # define matchingrecipes as finding the word in the list of recipes with some modifications*
    
    parsed_response = json.loads(json_resp)
    recipe_score = {}
    
    # apply tfidf --- read from offline file 
    tfidf = {}
    with open('all_stop_words.txt', 'r') as F:
        lines = F.readlines()
        for line in lines:
            words = line.replace('\n', '').split(' ')
            tfidf[words[0]] = int(words[1])
    
   # print(tfidf)
    
    for response in parsed_response:
        search_words = response['name'].lower().replace('-',' ').split(' ')
        #print('word in image :', search_words)
        for word in search_words:
            for ids,recipe_name in recipes.items():
                keywords = recipe_name.lower().replace('-',' ').split(' ')
                #print('recipe name is ', recipe_name)
                #print('keywords in recipe: ', keywords)
                if word in keywords:
                    #print('found word = ' , word, 'include ', recipe_name)
                    
                    word_score = 0
                    if(word in tfidf):
                        word_score = 1/tfidf[word]
                    else:
                        word_score = 1
                    
                    if(ids in recipe_score):
                        recipe_score[ids] += word_score
                    else:
                        recipe_score[ids] = word_score
                
                
        
    recipe_score = sorted(recipe_score.items(), key=lambda x: -x[1])
    return recipe_score

In [317]:
def predict_ids(imgpath, url_flag=1):
    with open('recipes.json', 'r') as F:
        json_recipes = json.load(F)

    recipes = {}
    for data in json_recipes['recipes']:
        recipes[data['id']] = data['name']

    #print(recipes)
    matching_ids = get_matching_recipes(get_ingreds(imgpath,url_flag), recipes, url_flag)
    return matching_ids[:5]  # return top 5

In [318]:
#predict_ids('images/turkey_cranberry.jpg',url_flag = 0)
#predict_ids('https://en.wikipedia.org/wiki/List_of_noodle_dishes#/media/File:Xing_hua_lor_mee.jpg',url_flag = 1)
#predict_ids('https://upload.wikimedia.org/wikipedia/commons/4/4b/Baby-back_ribs.jpg',url_flag = 1)

predict_ids('https://firebasestorage.googleapis.com/v0/b/ubhacking-221502.appspot.com/o/images%2FfoodImage.jpg?alt=media&token=941439d4-ef87-4d95-9fc9-72ecb118ec86',url_flag = 1)

{'id': 'ai_69gDDQgl', 'name': 'hamburger', 'value': 0.9412627220153809, 'app_id': 'main'}
{'id': 'ai_QLn2rxmZ', 'name': 'lettuce', 'value': 0.9400391578674316, 'app_id': 'main'}
{'id': 'ai_CB8hsS3T', 'name': 'tomato', 'value': 0.9363461136817932, 'app_id': 'main'}
{'id': 'ai_KF7Xcz9J', 'name': 'onion', 'value': 0.8969442844390869, 'app_id': 'main'}
{'id': 'ai_rzQV6Hv6', 'name': 'falafel', 'value': 0.8760938048362732, 'app_id': 'main'}
{'id': 'ai_25Wc02Fc', 'name': 'bun', 'value': 0.8486544489860535, 'app_id': 'main'}
{'id': 'ai_FnZCSVMH', 'name': 'cheese', 'value': 0.8410243391990662, 'app_id': 'main'}
{'id': 'ai_GC6FB0cQ', 'name': 'sauce', 'value': 0.8370276689529419, 'app_id': 'main'}
{'id': 'ai_SqL5hFw6', 'name': 'arugula', 'value': 0.8349764347076416, 'app_id': 'main'}
{'id': 'ai_XVpwLB09', 'name': 'beef', 'value': 0.8049886226654053, 'app_id': 'main'}
{'id': 'ai_H2d8CVB7', 'name': 'salad', 'value': 0.6971393823623657, 'app_id': 'main'}
{'id': 'ai_QrXHPz41', 'name': 'cumin', 'value

[(21406, 1.0), (5388, 0.4), (4858, 0.4), (13870, 0.325), (20815, 0.25)]